# Next TODO: 

Split code in parts!
I need a working deck, that I can learn with <b>as fast as possible!</b>

- <span style="color:lightgreen">Anki card styling!</span> Like 'the NewYorker' or 'Economist'! Black fonts, different sizes, like a newspaper! With thin lines between definition, hint, notes...
- add <span style="color:lightgreen">『』, "" </span> to long_sentence for better styling
- load <span style="color:lightgreen">"native_langage"</span> as a column and have a native_message for everything!  
- <span style="color:lightgreen">ふりがな</span> with a python library, not in Anki. Apparently 'Sudachi' is very reliable... Mecab. (kodachi) Have <span style="color:lightgreen">furigana</span> in separate code!
- maybe include a japanese dictionary like https://pypi.org/project/jamdict/ to improve the definitions. Which dictionaries should I rely on? API, downloaded html dictionary or something else? What does Lisardo say?

## Wish list: 
- more synonyms, always also in target_language
- related phrases, common phrases....
- create a gpt assistant that gives better definitions
- more Kanji information
Different information depending on word_type:
    + nouns: related words, びょういん and 病気　etc,
    + verbs: conjugation, て-Form! (to know if る or う), prepositions for the verbs...
    + adjectives: な or い, conjugation examples


In [7]:
import json
import genanki
import re
import pandas as pd
from dotenv import load_dotenv
import os
import openai
from jinja2 import Template

json_file_path = '/Users/moritzvitt/coding/jn_config.json'
csv_file_path = '/Users/moritzvitt/downloads/anki/items.csv'

# Open the JSON file and load its contents into a Python dictionary
with open(json_file_path, 'r') as json_file:
    config_json = json.load(json_file)

# chatGpt configuration
load_dotenv()
openai.api_key = os.getenv("22API_KEY")

load_dotenv()
ai_model = os.getenv("model")

# Define the column names of your CSV file
column_names = config_json['column_names']
df = pd.read_csv(csv_file_path, delimiter='\t', names=column_names)

# drop all rows where word_or_phrase is phrase
df = df[df['word_or_phrase'] != 'Phrase']

# native_language column

df['native_language'] = config_json['native_language']
# print(df.columns)
# print(df.word_type)
df.head(2)

NameError: name 'df_anki' is not defined

# creating the 'messages', to send to chatGPT, with a templating library

In [8]:
df_messages = pd.DataFrame()

def load_and_resolve_template(template_dict, template_string):
    template = Template(template_string)
    resolved_template = template.render(template_dict)
    return resolved_template

for message in config_json["messages"]:
    message_name = f"{message}"
    df_messages[message_name] = df.apply(lambda row: load_and_resolve_template(row.to_dict(), config_json["messages"][message_name]), axis=1)

def filter_fields(df_messages, wanted_fields):
    for field, value in wanted_fields.items():
        if not value:
            print(field)
            df_messages.drop(field, axis=1, inplace=True)
            
        else:
            pass
filter_fields(df_messages, config_json["wanted_fields"])

df_messages.head()
print(df_messages)

pd.set_option('display.max_columns', None)  # None means unlimited
pd.set_option('display.expand_frame_repr', False)  # Don't wrap to multiple lines
pd.set_option('display.max_colwidth', None)  # None means unlimited

print(df_messages)

NameError: name 'pd' is not defined

In [ ]:
# # delete the field "conjugation" for every row where the field "word_or_phrase" is 'Verb', but still keep the row, only delete this field
# # I only want to add the word_type field to the df_messages on index, maybe with concat?

# # df_messages = df_messages[df_messages['word_type'] != 'Verb']

# df_messages = pd.concat([df, df_messages], axis=1)
# print(df_messages)
# # df_messages = df.join(df_messages)

# df_messages = df_messages[df_messages['word_type'] != 'Verb']

# print(df_messages)

# Creating the 'content' with chatGPT

In [ ]:
df_anki = pd.DataFrame()
def generate_content(message):
    response = openai.ChatCompletion.create(
        model=ai_model,
        messages=[{"role": "user", "content": message}],
        max_tokens=500,
        # temperature=0.5
    )
    content = response.choices  [0].message["content"]
    print(content)
    return content

for column in df_messages.columns:
    df_anki[column] = df_messages.apply(lambda row: generate_content(row[column]), axis=1)

df_anki.head

df_anki = pd.concat([df_anki, df[['ID', 'word', 'audio', 'first_jpg', 'second_jpg', 'long_phrase', 'human_translation', 'word_translation', 'source', 'date']]], axis=1)
df_anki.head(2)

# programm a la

identical, same
murky, cloudy
dried fish, fish jerky
earthworm, worm
try, attempt
pull, draw
unoccupied, uncrowded
good, fine
investigate, examine
crept in, gotten inside
"but" and "however"
Hey, Stop
mugwort, Artemisia princeps
彼女は毎朝同じコーヒーショップでコーヒーを買います。
She buys coffee at the same coffee shop every morning.
彼女の心には濁った感情が渦巻いている。
Her heart is swirling with murky emotions.
私のおばあちゃんは、自家製の干物を作るのが上手です。
My grandmother is skilled at making homemade dried foods.
ミミズが土の中を這っている。
The earthworm is crawling in the soil.
これはやったことがない料理方法です。
This is a cooking method I've never tried before.
今日の市場は高値で引けた。
The market closed at a high today.
混雑した電車はとても不快です。
Crowded trains are very uncomfortable.
この本はとても良いです。
This book is very good.
日本の歴史について調べるのはとても興味深いです。
Researching about Japanese history is very interesting.
この本は面白いですね。<br>
This book is interesting, isn't it?
夏でもコートを着る人がいます。
Some people wear coats even in the summer.
こら、そんなに早く走ったら危ないよ。
Hey, it's dangerous to run that fast.
桜の木から花をもぎ取ってはいけません。
Do not p

,synonyms,first_example,second_example,hint,definition,grammar,ID,word,audio,first_jpg,second_jpg,long_phrase,human_translation,word_translation,source,date
0,"identical, same",彼女は毎朝同じコーヒーショップでコーヒーを買います。\nShe buys coffee at the same coffee shop every morning.,私たちは同じ本を読んでいます。\nWe are reading the same book.,同じ: means the same.,"同じ: Means ""the same"" or ""identical.""","こんだけ: これだけ, this much<br>\n濁ってりゃ: 濁っていれば, if it's muddy (濁る + て + いれば, contraction into りゃ)<br>\nこすらなくても: Even without scrubbing (こする + ない + ても)<br>\n同じだな: it's the same (同じ + だ = is, な = casual emphasis/particle)<br>",WORD|同じ|ja,同じ,1707416053694.mp3,1707416053694_prev.jpg,1707416053694_next.jpg,こいつにはさ ミミズの干物(ひもの)が入ってんだ\nこんだけ濁(にご)ってりゃ こすらなくても同じだな\nいっぱいになったら もう一回引きな 止まるから,"That's the dried worm salts.\nWith water this murky, who's gonna see the sludge?\nYank it again when it's full. It'll stop.","same, equal, identical",Spirited Away,2024-02-08 18:14
1,"murky, cloudy",彼女の心には濁った感情が渦巻いている。\nHer heart is swirling with murky emotions.,その水は濁っていて飲めない。\nThat water is muddy and undrinkable.,にごる: to be cloudy/muddy.,"濁る: To become muddy or unclear, as in liquid or situation.","こんだけ: これだけ(this much), colloquial contraction<br>\n濁(にご)ってりゃ: 濁る(to be muddy) + ている(present continuous) + りゃ(if, when), colloquial contraction, stands for 濁っていれば<br>\nこすらなくても: こする(to scrub) + ない (not) + ても(even if), 'even without scrubbing'<br>\n同じ: same<br>\nだな: casual assertion, だ(copula) + な(sentence-ending particle)<br>",WORD|濁る|ja,濁,1707416049695.mp3,1707416049695_prev.jpg,1707416049695_next.jpg,こいつにはさ ミミズの干物(ひもの)が入ってんだ\nこんだけ濁(にご)ってりゃ こすらなくても同じだな\nいっぱいになったら もう一回引きな 止まるから,"That's the dried worm salts.\nWith water this murky, who's gonna see the sludge?\nYank it again when it's full. It'll stop.","muddy, pollution, turbid",Spirited Away,2024-02-08 18:14


# Formatting hint, definition and notes fields!

In [ ]:
df_anki.head(2)

,synonyms,first_example,second_example,hint,definition,grammar,ID,word,audio,first_jpg,second_jpg,long_phrase,human_translation,word_translation,source,date
0,"identical, same",彼女は毎朝同じコーヒーショップでコーヒーを買います。\nShe buys coffee at the same coffee shop every morning.,私たちは同じ本を読んでいます。\nWe are reading the same book.,同じ: means the same.,"同じ: Means ""the same"" or ""identical.""","こんだけ: これだけ, this much<br>\n濁ってりゃ: 濁っていれば, if it's muddy (濁る + て + いれば, contraction into りゃ)<br>\nこすらなくても: Even without scrubbing (こする + ない + ても)<br>\n同じだな: it's the same (同じ + だ = is, な = casual emphasis/particle)<br>",WORD|同じ|ja,同じ,1707416053694.mp3,1707416053694_prev.jpg,1707416053694_next.jpg,こいつにはさ ミミズの干物(ひもの)が入ってんだ\nこんだけ濁(にご)ってりゃ こすらなくても同じだな\nいっぱいになったら もう一回引きな 止まるから,"That's the dried worm salts.\nWith water this murky, who's gonna see the sludge?\nYank it again when it's full. It'll stop.","same, equal, identical",Spirited Away,2024-02-08 18:14
1,"murky, cloudy",彼女の心には濁った感情が渦巻いている。\nHer heart is swirling with murky emotions.,その水は濁っていて飲めない。\nThat water is muddy and undrinkable.,にごる: to be cloudy/muddy.,"濁る: To become muddy or unclear, as in liquid or situation.","こんだけ: これだけ(this much), colloquial contraction<br>\n濁(にご)ってりゃ: 濁る(to be muddy) + ている(present continuous) + りゃ(if, when), colloquial contraction, stands for 濁っていれば<br>\nこすらなくても: こする(to scrub) + ない (not) + ても(even if), 'even without scrubbing'<br>\n同じ: same<br>\nだな: casual assertion, だ(copula) + な(sentence-ending particle)<br>",WORD|濁る|ja,濁,1707416049695.mp3,1707416049695_prev.jpg,1707416049695_next.jpg,こいつにはさ ミミズの干物(ひもの)が入ってんだ\nこんだけ濁(にご)ってりゃ こすらなくても同じだな\nいっぱいになったら もう一回引きな 止まるから,"That's the dried worm salts.\nWith water this murky, who's gonna see the sludge?\nYank it again when it's full. It'll stop.","muddy, pollution, turbid",Spirited Away,2024-02-08 18:14


In [ ]:
def add_fields(df_anki, wanted_fields):
    for field, should_add in wanted_fields.items():
        if not should_add:
            field_name = f"{field}"
            df_anki[field_name] = ""

add_fields(df_anki, config_json["wanted_fields"])

def clean_long_sentence(sentence):
    if "(" in sentence and ")" in sentence:
        cleaned_sentence = re.sub(r'\(.*?\)', '', sentence)
        return cleaned_sentence.strip()
    else:
        return sentence

df_anki["long_phrase"] = df_anki["long_phrase"].apply(clean_long_sentence)

import random

def createID(ID, source, date):
    newID = f"{ID}{source}{date}"
    newID = newID.replace(" ", "__")
    #random number
    newID = newID + "__" + str(random.randint(1000, 9999))
    return newID
df_anki["ID"] = df_anki.apply(lambda row: createID(row["ID"], row["source"], row["date"]), axis=1)

def mp3_pattern(mp3):
        mp3 = "[sound:" + mp3 + "]"
        return mp3
df_anki["audio"] = df_anki.apply(lambda row: mp3_pattern(row["audio"]), axis=1)

def image(first_jpg, second_jpg):
    image = f'<img src="{first_jpg}"><img src="{second_jpg}">'
    return image
df_anki["image"] = df_anki.apply(lambda row: image(row["first_jpg"], row["second_jpg"]), axis=1)

def clozePattern(word, phrase, synonyms):
    word_new = '{' + '{' f'c1::{word}::{synonyms}' + '}' + '}'
    print(word_new)

    phrase = re.sub(f'{word}', word_new, phrase, flags=re.IGNORECASE)
    print(phrase)
    return phrase

df_anki["cloze"] = df_anki.apply(lambda row: clozePattern(row["word"], row["long_phrase"], row["synonyms"]), axis=1)

def cleanHint(hint):
    # print(type(hint))
    if hint != "" and ":" in hint:
        parts = re.findall(r"[:](.+)?", hint) 
        # r"[:](.+)?" worked
        return f": {parts[0]}"
    else: 
        return ""
df_anki["hint"] = df_anki["hint"].apply(cleanHint)

def definition_field(definition, explanation, first_example, second_example, grammar):
    new_grammar = '<br>' + grammar 
    fields = [field for field in [definition, explanation, first_example, second_example, new_grammar] if field]
    newDefinition = "<br>".join(map(str, fields))
    return newDefinition

df_anki["definition"] = df_anki.apply(lambda row: definition_field(row["definition"], row["explanation"], row["first_example"], row["second_example"], row["grammar"]), axis=1)


def notes_field(translation, word_translation):
    translation_str = str(translation) if not pd.isnull(translation) else ""
    word_translation_str = str(word_translation) if not pd.isnull(word_translation) else ""
    newNotes = translation_str + "<br><br>" + word_translation_str + "<br>"
    return newNotes

df_anki["notes"] = df_anki.apply(lambda row: notes_field(row["human_translation"], row["word_translation"]), axis=1)

df_anki[["ID", "cloze", "hint", "definition", "image", "audio"]].head(2)

{{c1::同じ::identical, same}}
こいつにはさ ミミズの干物が入ってんだ
こんだけ濁ってりゃ こすらなくても{{c1::同じ::identical, same}}だな
いっぱいになったら もう一回引きな 止まるから
{{c1::濁::murky, cloudy}}
こいつにはさ ミミズの干物が入ってんだ
こんだけ{{c1::濁::murky, cloudy}}ってりゃ こすらなくても同じだな
いっぱいになったら もう一回引きな 止まるから
{{c1::干物::dried fish, fish jerky}}
うわあ すごい色
こいつにはさ ミミズの{{c1::干物::dried fish, fish jerky}}が入ってんだ
こんだけ濁ってりゃ こすらなくても同じだな
{{c1::ミミズ::earthworm, worm}}
うわあ すごい色
こいつにはさ {{c1::ミミズ::earthworm, worm}}の干物が入ってんだ
こんだけ濁ってりゃ こすらなくても同じだな
{{c1::やっ::try, attempt}}
混んでないから すぐ来るよ きっと
これを引けばお湯が出る {{c1::やっ::try, attempt}}てみな
（千尋）うわっ！
{{c1::引け::pull, draw}}
混んでないから すぐ来るよ きっと
これを{{c1::引け::pull, draw}}ばお湯が出る やってみな
（千尋）うわっ！
{{c1::混::unoccupied, uncrowded}}
これがさ 釜爺のとこへ行くんだ
{{c1::混::unoccupied, uncrowded}}んでないから すぐ来るよ きっと
これを引けばお湯が出る やってみな
{{c1::良::good, fine}}
それを調べるんだ 今日はハクがいないからね
へえ ずいぶん{{c1::良::good, fine}}いのくれたじゃん
これがさ 釜爺のとこへ行くんだ
{{c1::調べる::investigate, examine}}
（番台蛙）人間ですか？
それを{{c1::調べる::investigate, examine}}んだ 今日はハクがいないからね
へえ ずいぶん良いのくれたじゃん
{{c1::こ::crept in, gotten inside}}
い

,ID,cloze,hint,definition,image,audio
0,WORD|同じ|jaSpirited__Away2024-02-08__18:14__8797,"こいつにはさ ミミズの干物が入ってんだ\nこんだけ濁ってりゃ こすらなくても{{c1::同じ::identical, same}}だな\nいっぱいになったら もう一回引きな 止まるから",: means the same.,"同じ: Means ""the same"" or ""identical.""<br>彼女は毎朝同じコーヒーショップでコーヒーを買います。\nShe buys coffee at the same coffee shop every morning.<br>私たちは同じ本を読んでいます。\nWe are reading the same book.<br><br>こんだけ: これだけ, this much<br>\n濁ってりゃ: 濁っていれば, if it's muddy (濁る + て + いれば, contraction into りゃ)<br>\nこすらなくても: Even without scrubbing (こする + ない + ても)<br>\n同じだな: it's the same (同じ + だ = is, な = casual emphasis/particle)<br>","<img src=""1707416053694_prev.jpg""><img src=""1707416053694_next.jpg"">",[sound:1707416053694.mp3]
1,WORD|濁る|jaSpirited__Away2024-02-08__18:14__4118,"こいつにはさ ミミズの干物が入ってんだ\nこんだけ{{c1::濁::murky, cloudy}}ってりゃ こすらなくても同じだな\nいっぱいになったら もう一回引きな 止まるから",: to be cloudy/muddy.,"濁る: To become muddy or unclear, as in liquid or situation.<br>彼女の心には濁った感情が渦巻いている。\nHer heart is swirling with murky emotions.<br>その水は濁っていて飲めない。\nThat water is muddy and undrinkable.<br><br>こんだけ: これだけ(this much), colloquial contraction<br>\n濁(にご)ってりゃ: 濁る(to be muddy) + ている(present continuous) + りゃ(if, when), colloquial contraction, stands for 濁っていれば<br>\nこすらなくても: こする(to scrub) + ない (not) + ても(even if), 'even without scrubbing'<br>\n同じ: same<br>\nだな: casual assertion, だ(copula) + な(sentence-ending particle)<br>","<img src=""1707416049695_prev.jpg""><img src=""1707416049695_next.jpg"">",[sound:1707416049695.mp3]


In [ ]:
from sudachipy import tokenizer
from sudachipy import dictionary
from wanakana import to_hiragana, is_hiragana, is_japanese, is_katakana
import string

tokenizer_obj = dictionary.Dictionary(dict_type="small").create()

KANJI_READING_MAPPING = {
    '私': '私[わたし]',
    '貴女': '貴女[あなた]',
    '何': '何[なに]',
    '外宇宙': '外宇宙[がいうちゅう]',
    '異星人': '異星人[いせいじん]',
    '優那': '優那[ゆうな]',
    '菜々美': '菜々美[ななみ]'
}

JAPANESE_PUNCTUATION = '　〜！？。、（）：「」『』０１２３４５６７８９ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ'

SPECIAL_CHARACTERS = '〜'

def is_japanese_extended(text):
    return is_japanese(text) and text not in string.punctuation and text not in JAPANESE_PUNCTUATION

def to_anki_format(index, kanji, reading):
    return '{}{}[{}]'.format(' ' if index > 0 else '', kanji, reading) 

def add_furigana(text):
    tokens = [m for m in tokenizer_obj.tokenize(text, tokenizer.Tokenizer.SplitMode.C)]
    parsed = ''
    token_indexes_to_skip = []
    for index, token in enumerate(tokens):   
        if index in token_indexes_to_skip:
            continue
        to_parse = is_japanese_extended(token.surface()) and not is_katakana(token.surface()) and not is_hiragana(token.surface())
        if to_parse:
            if token.surface()[-1] in SPECIAL_CHARACTERS:
                parsed += add_furigana(token.surface()[:-1]) + token.surface()[-1]
            else:
                if index > 0:
                    parsed += ' '
                reading = to_hiragana(token.reading_form())
                if token.surface() in KANJI_READING_MAPPING:
                    parsed += KANJI_READING_MAPPING[token.surface()]
                elif index < len(tokens)-1 and token.surface() + tokens[index+1].surface() in KANJI_READING_MAPPING:
                    parsed += KANJI_READING_MAPPING[tokens[index].surface() + tokens[index+1].surface()]
                    token_indexes_to_skip.append(index+1)
                else:
                    surface_index = 0
                    reading_index = 0
                    while len(token.surface()) > surface_index:
                        if is_hiragana(token.surface()[surface_index]) or is_katakana(token.surface()[surface_index]):
                            parsed += token.surface()[surface_index]
                            reading_index += 1
                            surface_index += 1
                        else:
                            next_index = -1
                            for token_index in range(surface_index, len(token.surface())):
                                if is_hiragana(token.surface()[token_index]) or is_katakana(token.surface()[token_index]):
                                    next_index = token_index
                                    break
                            if next_index < 0:
                                parsed += to_anki_format(
                                  index=surface_index, 
                                  kanji=token.surface()[surface_index:], reading=reading[reading_index:])
                                break
                            else:
                                reading_index_tail = reading_index
                                while reading_index_tail < len(reading) and (reading[reading_index_tail] != token.surface()[next_index] or (reading_index_tail < len(reading)-1 and reading[reading_index_tail] == reading[reading_index_tail+1])):
                                    reading_index_tail += 1
                                if reading_index_tail < len(reading):
                                    parsed += to_anki_format(
                                        index=surface_index, 
                                        kanji=token.surface()[surface_index:next_index], reading=reading[reading_index:reading_index_tail])
                                    reading_index = reading_index_tail
                                else:
                                    # Handle out-of-range index
                                    parsed += to_anki_format(
                                        index=surface_index,
                                        kanji=token.surface()[surface_index:next_index], reading=reading[reading_index:])
                                    break
                            reading_length = next_index - surface_index
                            if reading_length > 0:
                                surface_index += reading_length
                            else:
                                break
        else:
            parsed += token.surface()
    return parsed


# test = ['私', 'その代わり その子を しっかり守るんだよ', 'Japanese', '可愛い','いい加減', '喧嘩', '咲く', '入り口', '兎に角', '取り逃す', 'とり戻す', '絞り込んだ', 'この先に多数の機械生命体反応を確認', '広いな……この廃墟。']
# for s in test:
#     print(add_furigana(s))

df_anki = df_anki.map(lambda x: add_furigana(x) if isinstance(x, str) else x)
df_anki.head


/var/folders/dy/67d_dwt15t93yftrvlycwhm40000gn/T/ipykernel_25689/3507510873.py:6: DeprecationWarning: Parameter dict_type of Dictionary() is deprecated, use dict instead
  tokenizer_obj = dictionary.Dictionary(dict_type="small").create()


<bound method NDFrame.head of                        synonyms                                                                                                           first_example                                                                                                                                                                        second_example                                   hint                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
# Read content from the external file
# read the df_anki into a csv file

# df_anki.to_csv('anki.csv', index=False, sep='\t', encoding='utf-8')



with open('anki.html', 'r', encoding='utf-8') as content_file:
    content = content_file.read()

# Splitting HTML content
html_sections = content.split('<!-- html -->')

# Assigning sections to qfmt, afmt, and css
qfmt_html = html_sections[1]
afmt_html = html_sections[2]
css_code = html_sections[3]

df = df_anki[["ID", "cloze", "hint", "definition", "notes", "image", "audio"]]
df.to_csv('anki.csv', index=False, sep='\t', encoding='utf-8')
# so no problem later with the anki package, to string
df = df.astype(str)

# Define the Anki model
model_id = 1607392319
model = genanki.Model(
    model_id,
    'Cloze Deletion Model',
    fields=[
        {'name': 'ID'},
        {'name': 'cloze'},
        {'name': 'hint'},
        {'name': 'definition'},
        {'name': 'notes'},
        {'name': 'image'},
        {'name': 'audio'},
    ],

    templates=[
        {
            'name': 'Card 1',
            'qfmt': qfmt_html,
            'afmt': afmt_html,
        },
    ],
    css=css_code
)

# Create an Anki deck
deck_id = model_id + 1  # Ensure deck_id is different from model_id
deck = genanki.Deck(deck_id, 'lln_anki_deck')

# Add cards to the deck
# Add cards to the deck
for index, row in df.iterrows():
    my_note = genanki.Note(
        model=model,
        fields=[(row['ID']), (row['cloze']), (row['hint']), (row['definition']), (row['notes']), (row['image']), (row['audio'])],
    )
    deck.add_note(my_note)


# Specify the desktop path
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")

# Create Anki package on the desktop
package_path = os.path.join(desktop_path, 'lln_anki_deck.apkg')
package = genanki.Package(deck)
package.write_to_file(package_path)
print(f'Anki package "{package_path}" has been created.')

Anki package "/Users/moritzvitt/Desktop/lln_anki_deck.apkg" has been created.
